**EDA**

**Tokenizer**

In [ ]:
import regex as re

GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

class Tokenizer:
    def __init__(self):

        self.lookup = {}


    def train(self, train, vocab_size, verbose=False):
        #Here encoding dictonary is being made

        token = re.compile(GPT4_SPLIT_PATTERN)
        token_list = re.findall(token, train)

        token_list.encode("UTF-8")